In [1]:
import sys
import os
import time
import pandas as pd
import pathlib
tutorial_dir = pathlib.Path().resolve()
# Importing the main code.
module_path = os.path.abspath(os.path.join('..', 'src', 'ocloc'))
if module_path not in sys.path:
    sys.path.append(module_path)
from ocloc import ProcessingParameters, ClockDrift
from ocloc import read_correlation_file

In [4]:
# Parameters for locating the files where the correlation files and station 
# information is contained.
path2data_dir = "/Users/localadmin/Dropbox/GitHub/data"
# The reference time is zero time to which the dates of the correlations
# will be compared. It can be the date of deployment. 
reference_time = '2014-08-21T00:00:00.000000Z'

station_file = os.path.join(tutorial_dir, "station_info")
params2 = ProcessingParameters(
                 freqmin = 0.2, # Low freq. for the bandpass filter
                 freqmax = 0.4, # High freq. for the bandpass filter 
                 ref_vel = 4500, # m/s
                 dist_trh = 2.5, # Minimum station separation in terms of wavelength
                 snr_trh = 30, # Signal-to-noise ratio threshold
                 noise_st = 240, # start of the noise window.
                 dt_err = 0.004, # Sampling interval needs to be multiple of this value.
                 resp_details = False)
cd = ClockDrift(station_file, path2data_dir, 
                  reference_time = '2014-08-21T00:00:00.000000Z',
                  list_of_processing_parameters=[params2])#, params3])

No correlation file found for station:O26


In [5]:
cd.calculate_aprioridt_4_allcorrelations_corrected()
cd.calculate_dt_ins()
#cd.calculate_tapp_4_allcorrelations()

Calculating the apriori estimates for each stationpair


In [14]:
import numpy as np
import obspy
self = cd.correlations[119]
# for i, self in enumerate(cd.correlations):
freqmin = self.processing_parameters.freqmin
freqmax = self.processing_parameters.freqmax
ref_vel = self.processing_parameters.ref_vel
dist_trh = self.processing_parameters.dist_trh
snr_trh = self.processing_parameters.snr_trh
noise_st = self.processing_parameters.noise_st
dt_err = self.processing_parameters.dt_err
xcorr_path = self.file_path
v1 = 2 * ref_vel # Highest vel. for finding the interferometric responses.
v2 = ref_vel / 2.0 # Lowest velocity for finding the interferometric responses.
central_freq = (freqmax - freqmin) / 2.0

# Now we load the trace.
tr = read_correlation_file(xcorr_path)
nn = tr.stats.npts # Number of samples.
dt = tr.stats.delta

nsmpls_per_hp= round(1/central_freq/2/dt) #! Number of samples per half-period
sig = round(1/central_freq/3/dt) # Signal
sig_ext=2*sig
sig_int=round(sig*(dt/dt_err))
sig_int_ext=2*sig_int

# Station separation. Great circle distance in m using WGS84
# ellipsoid.
cpl_dist = self.cpl_dist
min_wl = ref_vel / freqmax  # Minimum wavelength separation.

if cpl_dist / min_wl < dist_trh:
    msg = "Station couple does not exceed minimum separation"
    self.resp_details = msg
    if not isinstance(self.t_app, list):
        self.t_app = [np.nan]
    else:
        self.t_app.append(np.nan)
    
    print(msg) #return (np.nan, [msg, cpl_dist])
try:
    # Apriori estimates of both stations.
    apr_dt_st1 = float(self.dt_ins_station1[-1])
    apr_dt_st2 = float(self.dt_ins_station2[-1])
except:
    msg = "No apriori estimate found for station 1 and station 2."
    raise Exception(msg)

tr = tr.taper(max_percentage=0.1, type="cosine")
tr = tr.filter("bandpass", freqmin=freqmin, freqmax=freqmax)

apr_dt_cpl = 0
apr_dt_cpl = apr_dt_cpl + (2 * apr_dt_st1)
apr_dt_cpl=apr_dt_cpl - (2 * apr_dt_st2)
nt = nn / 2 # Dividing no. of samples in two.
def SNRwith_shift(tr, nt, cpl_dist, apr_dt_cpl, dt, noise_st, v1, v2):
    """ 
    """

    nt = nt + 1
    apr_dt_cpl = apr_dt_cpl / 2
    a1r = cpl_dist /v1+apr_dt_cpl
    a2r=cpl_dist/v2+apr_dt_cpl               # Ending time of causal signal window 
    b1r=noise_st+apr_dt_cpl              # Starting time of causal noise window
    b2r=2.0*noise_st+apr_dt_cpl               # Ending time of causal noise window 
    
    c_a1i=int(nt+round(a1r/dt))        # Starting index of causal signal window 
    c_a2i=int(nt+round(a2r/dt))        # Ending index of causal signal window
    c_b1i=int(nt+round(b1r/dt))        # Starting index of causal noise window 
    c_b2i=int(nt+round(b2r/dt))       # Ending index of acausal noise window

    a1r=cpl_dist/v1-apr_dt_cpl                # Starting time of acausal signal window 
    a2r=cpl_dist/v2-apr_dt_cpl                # Ending time of acausal signal window 
    b1r=noise_st-apr_dt_cpl              # Starting time of acausal noise window
    b2r=2.0*noise_st-apr_dt_cpl               # Ending time of acausal noise window 

    ac_a1i=int(nt-round(a1r/dt))       # Ending index of acausal signal window 
    ac_a2i=int(nt-round(a2r/dt) )      # Starting index of acausal signal window
    ac_b1i=int(nt-round(b1r/dt))       # Ending index of acausal noise window 
    ac_b2i=int(nt-round(b2r/dt))       # Starting index of acausal noise window

    a1r=cpl_dist/v1-apr_dt_cpl       #         ! Starting time of acausal signal window 
    a2r=cpl_dist/v2-apr_dt_cpl        #        ! Ending time of acausal signal window 
    b1r=noise_st-apr_dt_cpl            #   ! Starting time of acausal noise window
    b2r=2.0*noise_st-apr_dt_cpl         #       ! Ending time of acausal noise window 

    ac_a1i=int(nt-round(a1r/dt))  #      ! Ending index of acausal signal window 
    ac_a2i=int(nt-round(a2r/dt))   #     ! Starting index of acausal signal window
    ac_b1i=int(nt-round(b1r/dt))    #    ! Ending index of acausal noise window 
    ac_b2i=int(nt-round(b2r/dt) )    #   ! Starting index of acausal noise window
    c_amp=max(abs(tr.data[c_a1i:c_a2i]))
    ac_amp= max(abs(tr.data[ac_a2i:ac_a1i])) 
    c_sum= sum( tr.data[c_b1i:c_b2i])
    ac_sum=sum(tr.data[ac_b2i:ac_b1i])
    tsum=c_sum+ac_sum
    n=(c_b2i-c_b1i+1)+(ac_b1i-ac_b2i+1)
    mean=tsum / n
    var=0.
    for i in range(c_b1i, c_b2i):
        var=var+(((tr[i])-mean)**2)
    
    for i in range(ac_b2i, ac_b1i):
        var=var+(((tr[i])-mean)**2)

    nvar=var/(n) 
    std=np.sqrt(nvar)
    c_snr=c_amp/std
    ac_snr=ac_amp/std
    return c_snr, ac_snr

c_snr, ac_snr = SNRwith_shift(tr, nt, cpl_dist, apr_dt_cpl, dt, noise_st, v1, v2)
# Cycle if SNR's are not high enough at this frequncy
if(c_snr < snr_trh or ac_snr < snr_trh):
    msg = "Station couple does not exceed SNR"
    #raise Exception(msg)
else:
    print(i)
    

119


In [15]:
print(c_snr)
print(snr_trh)
ac_snr

39.2919314421
30.0


95.389077595286096

In [16]:
cor = cd.correlations[119]
cor.calculate_t_app()
tr.stats

mkdir: temp/: File exists
###############     PROGRAM COMPLETE      ################
'
 Results saved in folder:temp/EIN_O10_EIN_O10_1413530093_099.sac/
 Result shift:   0.79200000000000004     
 acausal signal from index:        44821
 acausal signal until index:        44571
 causal signal from index:        45181
 causal signal until index:        45431
 SNR causal wave   76.856708666895756     
 SNR acausal wave   94.853297429564449     
'


         network: 
         station: 
        location: 
         channel: 
       starttime: 1969-12-31T23:30:00.000000Z
         endtime: 1970-01-01T00:29:59.960000Z
   sampling_rate: 25.0
           delta: 0.04
            npts: 90000
           calib: 1.0
         _format: SAC
    average_date: 2014-10-17T07:14:53.000000Z
  number_of_days: 99.0
      processing: ["ObsPy 1.2.2: taper(max_length=None::max_percentage=0.1::side='both'::type='cosine')", "ObsPy 1.2.2: filter(options={'freqmin': 0.2, 'freqmax': 0.4}::type='bandpass')"]
             sac: AttribDict({'delta': 0.039999999, 'depmin': -296.46793, 'depmax': 218.02748, 'b': -3600.0, 'e': -0.040080465, 'depmen': 3.100124e-05, 'nzyear': 1970, 'nzjday': 1, 'nzhour': 0, 'nzmin': 30, 'nzsec': 0, 'nzmsec': 0, 'nvhdr': 6, 'npts': 90000, 'iftype': 1, 'leven': 1, 'lpspol': 0, 'lovrok': 1, 'lcalda': 1, 'unused23': 0, 'kevnm': ''})
    station_pair: EIN_O10